In [129]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import recall_score, precision_score, roc_auc_score

from catboost import CatBoostClassifier

from features import create_features

import warnings
warnings.filterwarnings("ignore")

In [2]:
TARGETS = [
    'Артериальная гипертензия',
    'ОНМК',
    'Стенокардия, ИБС, инфаркт миокарда',
    'Сердечная недостаточность',
    'Прочие заболевания сердца',
]

In [3]:
raw_path = '../data/raw/'
train = pd.read_csv(os.path.join(raw_path, 'train.csv'))
test = pd.read_csv(os.path.join(raw_path, 'test.csv'))

In [4]:
train_features = create_features(train, True)

test_features = create_features(test, False)

In [5]:
train_features.head()

,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца,Пол - М,Семья - в браке в настоящее время,Семья - в разводе,Семья - вдовец / вдова,Семья - гражданский брак / проживание с партнером,...,Регулярный прием лекарственных средств без болезней из опроса,Больной,Травмы / переломы,Идеальное здоровье,Без вредных привычек,Бросил вредную привычку,Макс. возраст вредной привычки,Само совершенство,Активное или пассивное курение,Продолжительность сна
ID,,,,,,,,,,,,,,,,,,,,,
54-102-358-02,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,18.0,0.0,1.0,8.0
54-103-101-01,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0
54-501-026-03,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,17.0,0.0,1.0,8.0
54-501-094-02,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,13.0,0.0,1.0,8.0
54-503-022-01,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,16.0,0.0,1.0,7.0


In [6]:
test_features.head(3)

,Пол - М,Семья - в браке в настоящее время,Семья - в разводе,Семья - вдовец / вдова,Семья - гражданский брак / проживание с партнером,Семья - никогда не был(а) в браке,Семья - раздельное проживание (официально не разведены),Семья - живет один,Этнос - европейская,Этнос - другая азиатская,...,Регулярный прием лекарственных средств без болезней из опроса,Больной,Травмы / переломы,Идеальное здоровье,Без вредных привычек,Бросил вредную привычку,Макс. возраст вредной привычки,Само совершенство,Активное или пассивное курение,Продолжительность сна
ID,,,,,,,,,,,,,,,,,,,,,
54-001-019-01,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,23.0,0.0,0.0,7.5
54-002-133-01,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,9.0
54-001-007-01,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,8.5


# Метрика

Нам нужно отпределить 5 типов заболеваемости у каждого больного: 
- 'Артериальная гипертензия'
- 'ОНМК'
- 'Стенокардия, ИБС, инфаркт миокарда'
- 'Сердечная недостаточность'
- 'Прочие заболевания сердца'

Так как у нас нет приоритета на каждую из них, мы будем считать метрики по каждой отдельно, а также среднюю метрику по всем.
В качестве метрик предлагаю взять полноту и точность, так как они достаточно хорошо опишут как хорошо наша модель может отличать больного той или иной болезнью от здорового, а также, сколько больных мы смогли определить из всей выборки.

В нашем случае нам больше будет важна метрика Recall, так как лучше мы лишний раз удостоверимся, что пациент не болеет, чем допустим болезнь. Т.е. мы хотим минимизировать ошибку II рода - сказать, что человек не болен при том, что он болен.

Нашей целевой метрикой будет модифицированная $F$-мера, с уклоном в сторону полноты, так как мы ориентируемся на нее.

Итоговая формула:

$5\cdot \frac{Precision + Recall}{(4\cdot Precision) + Recall} \rightarrow max$

In [307]:
def main_metric(y_real, y_pred):
    rec = recall_score(y_real, y_pred)
    prec = precision_score(y_real, y_pred)
    
    return 5*(prec*rec)/(4*prec + rec)

# Baseline

Найдем baseline предсказаний различных дефолтных моделей, чтоб понять, что наша финальная модель будет лучше

In [308]:
X_cols = list(test_features.columns)

In [309]:
scores_all_lr = []
scores_all_rf = []
scores_all_cb = []

scores_lr = {}
scores_rf = {}
scores_cb = {}
for target in TARGETS:
    scores_lr[target] = []
    scores_rf[target] = []
    scores_cb[target] = []

for i in tqdm_notebook(range(25)):
    
    for target in TARGETS:
        
        df_train, df_test = train_test_split(train_features, 
                                             test_size=0.3, 
                                             shuffle=True, 
                                             stratify=train_features[target])
        
        lr = LogisticRegression()
        rf = RandomForestClassifier()
        cb = CatBoostClassifier(verbose=False)
        
        lr.fit(df_train[X_cols], df_train[target])
        rf.fit(df_train[X_cols], df_train[target])
        cb.fit(df_train[X_cols], df_train[target])
        
        y_pred_lr = lr.predict(df_test[X_cols])
        y_pred_rf = rf.predict(df_test[X_cols])
        y_pred_cb = cb.predict(df_test[X_cols])
        
        score_lr = main_metric(df_test[target], y_pred_lr)
        score_rf = main_metric(df_test[target], y_pred_rf)
        score_cb = main_metric(df_test[target], y_pred_cb)
        
        if np.isnan(score_lr):
            score_lr = 0
        if np.isnan(score_rf):
            score_rf = 0
        if np.isnan(score_cb):
            score_cb = 0
        
        scores_all_lr.append(score_lr)
        scores_all_rf.append(score_rf)
        scores_all_cb.append(score_cb)
        
        scores_lr[target].append(score_lr)
        scores_rf[target].append(score_rf)
        scores_cb[target].append(score_cb)

lr_data = []
rf_data = []
cb_data = []
for target in TARGETS:
    lr_data.append(round(np.mean(scores_lr[target]),3))
    rf_data.append(round(np.mean(scores_rf[target]),3))
    cb_data.append(round(np.mean(scores_cb[target]),3))
    
lr_data.append(round(np.mean(scores_all_lr),3))
rf_data.append(round(np.mean(scores_all_rf),3))
cb_data.append(round(np.mean(scores_all_cb),3))

  0%|          | 0/25 [00:00<?, ?it/s]

In [310]:
indexes = []
for target in TARGETS:
    indexes.append(f'Score - {target}')
    
indexes.append('Average Score ALL')


df_res = pd.DataFrame.from_dict({'LogReg' : lr_data,
                                 'RandFor' : rf_data,
                                 'CatBoost' : cb_data})

df_res.index = indexes

df_res

,LogReg,RandFor,CatBoost
Score - Артериальная гипертензия,0.702,0.708,0.718
Score - ОНМК,0.016,0.000,0.004
"Score - Стенокардия, ИБС, инфаркт миокарда",0.052,0.007,0.004
Score - Сердечная недостаточность,0.034,0.007,0.000
Score - Прочие заболевания сердца,0.000,0.000,0.000
Average Score ALL,0.161,0.144,0.145


Как видим, пока что CatBoost имеет средние метрики. Предполагаю, что это происходит из-за большого количества фичей, которые не вносят никакой вклад, что и сказывается на случайном лесе. Именно поэтому у логистической регрессии результат лучше.

Отберем фичи для каждого типа таргета с помощью случайного леса.

# Отбор фичей

In [311]:
features_per_target = {}

for target in tqdm_notebook(TARGETS):
    
    print(f'Target - {target}')
    print('--------------------------')
    
    target_best_score = -np.inf
    best_features = []
    
    rf = RandomForestClassifier(n_estimators=600)
    
    rf.fit(train_features[X_cols], train_features[target])
    
    feature_importances = [0.0001, 0.001, 0.005, 0.01, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.5, 1]
    
    for f_imp in feature_importances:
        if np.max(rf.feature_importances_) < f_imp:
            continue
            
        new_features = []
        
        for col, val in zip(X_cols, rf.feature_importances_):
            if val > f_imp:
                new_features.append(col)
                
        
        scores = []
        
        for i in range(10):
            
            df_train, df_test = train_test_split(train_features, 
                                                 test_size=0.3, 
                                                 shuffle=True, 
                                                 stratify=train_features[target])

            cb = CatBoostClassifier(verbose=False)
            cb.fit(df_train[new_features], df_train[target])
            y_pred_cb = cb.predict(df_test[new_features])
            score_cb = main_metric(df_test[target], y_pred_cb)

            if np.isnan(score_cb):
                score_cb = 0

            scores.append(score_cb)
            
        
        mean_score = np.mean(scores)
        print(f'For feature importance {f_imp}')
        print(f'Score - {round(mean_score,3)}')
        print()
        
        if mean_score > target_best_score:
            target_best_score = mean_score
            best_features = new_features
            
        
    features_per_target[target] = best_features

  0%|          | 0/5 [00:00<?, ?it/s]

Target - Артериальная гипертензия
--------------------------
For feature importance 0.0001
Score - 0.713

For feature importance 0.001
Score - 0.705

For feature importance 0.005
Score - 0.695

For feature importance 0.01
Score - 0.668

For feature importance 0.05
Score - 0.8

For feature importance 0.06
Score - 0.81

For feature importance 0.07
Score - 0.814

Target - ОНМК
--------------------------
For feature importance 0.0001
Score - 0.01

For feature importance 0.001
Score - 0.02

For feature importance 0.005
Score - 0.0

For feature importance 0.01
Score - 0.0

For feature importance 0.05
Score - 0.0

For feature importance 0.06
Score - 0.0

For feature importance 0.07
Score - 0.0

For feature importance 0.08
Score - 0.0

For feature importance 0.09
Score - 0.0

Target - Стенокардия, ИБС, инфаркт миокарда
--------------------------
For feature importance 0.0001
Score - 0.004

For feature importance 0.001
Score - 0.004

For feature importance 0.005
Score - 0.011

For feature impor

Посмотрим скор дефолтной модели с отобраными фичами

In [312]:
scores_cb2 = []

for target in tqdm_notebook(TARGETS):
    
    scores = []
    for i in range(20):
        
        features_to_use = features_per_target[target]
        
        df_train, df_test = train_test_split(train_features, 
                                                 test_size=0.3, 
                                                 shuffle=True, 
                                                 stratify=train_features[target])

        cb = CatBoostClassifier(verbose=False)
        cb.fit(df_train[features_to_use], df_train[target])
        
        y_pred_cb = cb.predict(df_test[features_to_use])
        score_cb = main_metric(df_test[target], y_pred_cb)

        if np.isnan(score_cb):
            score_cb = 0

        scores.append(score_cb)
        
    scores_cb2.append(round(np.mean(scores), 3))

  0%|          | 0/5 [00:00<?, ?it/s]

In [313]:
scores_cb2.append(np.mean(scores_cb2))

In [314]:
indexes = []
for target in TARGETS:
    indexes.append(f'Score - {target}')
    
indexes.append('Average Score ALL')


df_res = pd.DataFrame.from_dict({'LogReg' : lr_data,
                                 'RandFor' : rf_data,
                                 'CatBoost' : cb_data,
                                 'CatBoost + features' : scores_cb2})

df_res.index = indexes

df_res

,LogReg,RandFor,CatBoost,CatBoost + features
Score - Артериальная гипертензия,0.702,0.708,0.718,0.809
Score - ОНМК,0.016,0.000,0.004,0.005
"Score - Стенокардия, ИБС, инфаркт миокарда",0.052,0.007,0.004,0.019
Score - Сердечная недостаточность,0.034,0.007,0.000,0.010
Score - Прочие заболевания сердца,0.000,0.000,0.000,0.007
Average Score ALL,0.161,0.144,0.145,0.170


Замечательно. Результаты уже стали сильно лучше.

Это выглядит как хороший baseline, который мы будем стараться побивать

# Model tuning

По заданию мы должны использовать Catboost, поэтому будем тюнить его

In [61]:
categorial_cols = ['Пол - М',
 'Семья - в браке в настоящее время',
 'Семья - в разводе',
 'Семья - вдовец / вдова',
 'Семья - гражданский брак / проживание с партнером',
 'Семья - никогда не был(а) в браке',
 'Семья - раздельное проживание (официально не разведены)',
 'Семья - живет один',
 'Этнос - европейская',
 'Этнос - другая азиатская',
 'Этнос - прочее',
 'Национальность - Азербайджанцы',
 'Национальность - Армяне',
 'Национальность - Башкиры',
 'Национальность - Белорусы',
 'Национальность - Буряты',
 'Национальность - Другие национальности',
 'Национальность - Евреи',
 'Национальность - Казахи',
 'Национальность - Киргизы',
 'Национальность - Лезгины',
 'Национальность - Молдаване',
 'Национальность - Мордва',
 'Национальность - Немцы',
 'Национальность - Русские',
 'Национальность - Таджики',
 'Национальность - Татары',
 'Национальность - Удмурты',
 'Национальность - Украинцы',
 'Национальность - Чуваши',
 'Национальность - Эстонцы',
 'Национальность - Азия / Кавказ',
 'Национальность - Русские / Белорусы / Украинцы',
 'Религия - Атеист / агностик',
 'Религия - Другое',
 'Религия - Индуизм',
 'Религия - Ислам',
 'Религия - Нет',
 'Религия - Христианство',
 'Религия - Индуизм / Ислам / Другое',
 'Религия - Верующий',
 'Образование - начальная школа',
 'Образование - среднее',
 'Образование - профессиональное училище',
 'Образование - ВУЗ',
 'Профессия - ведение домашнего хозяйства',
 'Профессия - вооруженные силы',
 'Профессия - дипломированные специалисты',
 'Профессия - квалифицированные работники сельского хозяйства и рыболовного',
 'Профессия - низкоквалифицированные работники',
 'Профессия - операторы и монтажники установок и машинного оборудования',
 'Профессия - законотворцы / менеджеры',
 'Профессия - сфера обслуживания',
 'Профессия - ремесленники и представители других отраслей промышленности',
 'Профессия - служащие',
 'Профессия - техники и младшие специалисты',
 'Профессия - военные / служащие / управление',
 'Профессия - промышленность',
 'Профессия - низкая квалификация',
 'Вы работаете?',
 'Выход на пенсию',
 'Прекращение работы по болезни',
 'Сахарный диабет',
 'Гепатит',
 'Онкология',
 'Хроническое заболевание легких',
 'Бронхиальная астма',
 'Туберкулез легких',
 'ВИЧ/СПИД',
 'Регулярный прием лекарственных средств',
 'Травмы за год',
 'Переломы',
 'Статус Курения - Бросил(а)',
 'Статус Курения - Курит',
 'Статус Курения - Никогда не курил(а)',
 'Пассивное курение',
 'Алкоголь - никогда не употреблял',
 'Алкоголь - ранее употреблял',
 'Алкоголь - употребляю в настоящее время',
 'Сон после обеда',
 'Спорт клубы',
 'Религия клубы',
 'Образование - Уровень',
 'Одинокий мужчина',
 'Одинокая женщина',
 'Без работы и не на пенсии',
 'Болезнь легких',
 'Инфекционная болезнь',
 'Хроническая болезнь',
 'Регулярный прием лекарственных средств без болезней из опроса',
 'Больной',
 'Травмы / переломы',
 'Идеальное здоровье',
 'Без вредных привычек',
 'Бросил вредную привычку',
 'Макс. возраст вредной привычки',
 'Само совершенство',
 'Активное или пассивное курение']

In [64]:
for col in categorial_cols:
    train_features[col] = train_features[col].astype(int)

## Артериальная гипертензия

In [318]:
target = 'Артериальная гипертензия'

In [319]:
using_features = features_per_target[target]

### Поиск по сетке

In [217]:
categorial_cols_using = [x for x in using_features if x in categorial_cols]

#Instantiate CatBoostClassifier
cbc = CatBoostClassifier(verbose=False, 
                         cat_features=categorial_cols_using,
                         )

cv = StratifiedKFold(n_splits=4, shuffle=True)

#create the grid
grid = {#'max_depth': [5,6,7,8,9,10],
        'n_estimators':[300,400,500,600],
        'min_child_samples':[4,8,12,16,20],
        'l2_leaf_reg':[0.5,1,5],
        'learning_rate':[0.005,0.01,0.05,0.1]}

#Instantiate GridSearchCV
gscv = GridSearchCV(estimator = cbc, param_grid = grid, scoring ='roc_auc', cv = cv, n_jobs=-1)

#fit the model
gscv.fit(df_train[using_features], df_train[target])

#returns the estimator with the best performance
print(gscv.best_estimator_)

#returns the best score
print(gscv.best_score_)

#returns the best parameters
print(gscv.best_params_)

0.7267224098555879
{'l2_leaf_reg': 0.5, 'learning_rate': 0.005, 'min_child_samples': 4, 'n_estimators': 300}


### Подбор трешхолда

In [320]:
thresholds = np.arange(0.1,1,0.1)
score_per_threshold = []

for thold in tqdm_notebook(thresholds):

    scores = []
    for i in range(10):

        features_to_use = features_per_target[target]

        df_train, df_test = train_test_split(train_features, 
                                                 test_size=0.3, 
                                                 shuffle=True, 
                                                 stratify=train_features[target])

        cb = CatBoostClassifier(max_depth=1,
                                n_estimators=300,
                                min_child_samples=4,
                                l2_leaf_reg=0.5,
                                learning_rate=0.005,
                                verbose=False)
        cb.fit(df_train[features_to_use], df_train[target])

        cb.set_probability_threshold(thold)

        y_pred_cb = cb.predict(df_test[features_to_use])
        score_cb = main_metric(df_test[target], y_pred_cb)

        if np.isnan(score_cb):
            score_cb = 0

        scores.append(score_cb)

    #print(round(np.mean(scores),3))
    
    score_per_threshold.append(round(np.mean(scores),3))
    
pd.DataFrame.from_dict({'threshold' : thresholds,
                        'scores' : score_per_threshold})

  0%|          | 0/9 [00:00<?, ?it/s]

,threshold,scores
0,0.1,0.814
1,0.2,0.814
2,0.3,0.812
3,0.4,0.813
4,0.5,0.819
5,0.6,0.807
6,0.7,0.000
7,0.8,0.000
8,0.9,0.000


### Итог

In [227]:
features_target_1 = ['Регулярный прием лекарственных средств']


target_1_model_params = {'max_depth':1,
                         'n_estimators':300,
                         'min_child_samples':4,
                         'l2_leaf_reg':0.5,
                         'learning_rate':0.005,
                         'verbose':False}

threshold = 0.2

## ОНМК

Ужасный признак, но мы не будем терять надежды)

In [335]:
target = 'ОНМК'

In [336]:
using_features = features_per_target[target]

### Поиск по сетке

In [250]:
categorial_cols_using = [x for x in using_features if x in categorial_cols]

#Instantiate CatBoostClassifier
cbc = CatBoostClassifier(verbose=False, 
                         cat_features=categorial_cols_using,
                         )

cv = StratifiedKFold(n_splits=3, shuffle=True)

#create the grid
grid = {'max_depth': [5,6,7,8,9,10],
        'n_estimators':[300,400,500,600],
        'min_child_samples':[4,8,12,16,20],
        'l2_leaf_reg':[0.5,1,5],
        'learning_rate':[0.005,0.01,0.05,0.1]}

#Instantiate GridSearchCV
gscv = GridSearchCV(estimator = cbc, param_grid = grid, scoring ='roc_auc', cv = cv, n_jobs=2)

#fit the model
gscv.fit(df_train[using_features], df_train[target])

#returns the estimator with the best performance
print(gscv.best_estimator_)

#returns the best score
print(gscv.best_score_)

#returns the best parameters
print(gscv.best_params_)

0.7397865178492338
{'l2_leaf_reg': 5, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_samples': 4, 'n_estimators': 500}


### Подбор трешхолда

In [337]:
thresholds = np.arange(0.01,0.4,0.05)
score_per_threshold = []

for thold in tqdm_notebook(thresholds):

    scores = []
    for i in range(10):

        features_to_use = features_per_target[target]

        df_train, df_test = train_test_split(train_features, 
                                                 test_size=0.3, 
                                                 shuffle=True, 
                                                 stratify=train_features[target])

        cb = CatBoostClassifier(max_depth=5,
                                n_estimators=500,
                                min_child_samples=4,
                                l2_leaf_reg=5,
                                learning_rate=0.1,
                                verbose=False)
        cb.fit(df_train[features_to_use], df_train[target])

        cb.set_probability_threshold(thold)

        y_pred_cb = cb.predict(df_test[features_to_use])
        score_cb = main_metric(df_test[target], y_pred_cb)

        if np.isnan(score_cb):
            score_cb = 0

        scores.append(score_cb)

    #print(round(np.mean(scores),3))
    
    score_per_threshold.append(round(np.mean(scores),3))
    
pd.DataFrame.from_dict({'threshold' : thresholds,
                        'scores' : score_per_threshold})

  0%|          | 0/8 [00:00<?, ?it/s]

,threshold,scores
0,0.01,0.220
1,0.06,0.142
2,0.11,0.078
3,0.16,0.072
4,0.21,0.082
5,0.26,0.066
6,0.31,0.067
7,0.36,0.039


### Итог

In [227]:
features_target_2 = ['Пол - М',
                     'Семья - в браке в настоящее время',
                     'Семья - вдовец / вдова',
                     'Семья - живет один',
                     'Национальность - Эстонцы',
                     'Образование - среднее',
                     'Образование - профессиональное училище',
                     'Профессия - дипломированные специалисты',
                     'Профессия - низкоквалифицированные работники',
                     'Профессия - ремесленники и представители других отраслей промышленности',
                     'Профессия - промышленность',
                     'Профессия - низкая квалификация',
                     'Вы работаете?',
                     'Выход на пенсию',
                     'Прекращение работы по болезни',
                     'Сахарный диабет',
                     'Регулярный прием лекарственных средств',
                     'Переломы',
                     'Статус Курения - Курит',
                     'Статус Курения - Никогда не курил(а)',
                     'Пассивное курение',
                     'Алкоголь - ранее употреблял',
                     'Алкоголь - употребляю в настоящее время',
                     'Сон после обеда',
                     'Спорт клубы',
                     'Возраст курения',
                     'Сигарет в день',
                     'Возраст алког',
                     'Образование - Уровень',
                     'Одинокая женщина',
                     'Без работы и не на пенсии',
                     'Сигарет в день (сейчас)',
                     'Частота пасс кур',
                     'Хроническая болезнь',
                     'Регулярный прием лекарственных средств без болезней из опроса',
                     'Травмы / переломы',
                     'Бросил вредную привычку',
                     'Макс. возраст вредной привычки',
                     'Активное или пассивное курение',
                     'Продолжительность сна']


target_2_model_params = {'max_depth':5,
                         'n_estimators':500,
                         'min_child_samples':4,
                         'l2_leaf_reg':5,
                         'learning_rate':0.1,
                         'verbose':False}

## Стенокардия, ИБС, инфаркт миокарда

In [338]:
target = 'Стенокардия, ИБС, инфаркт миокарда'

In [339]:
using_features = features_per_target[target]

### Поиск по сетке

In [253]:
categorial_cols_using = [x for x in using_features if x in categorial_cols]

#Instantiate CatBoostClassifier
cbc = CatBoostClassifier(verbose=False, 
                         cat_features=categorial_cols_using,
                         )

cv = StratifiedKFold(n_splits=3, shuffle=True)

#create the grid
grid = {'max_depth': [5,6,7,8,9,10],
        'n_estimators':[300,400,500,600],
        'min_child_samples':[4,8,12,16,20],
        'l2_leaf_reg':[0.5,1,5],
        'learning_rate':[0.005,0.01,0.05,0.1]}

#Instantiate GridSearchCV
gscv = GridSearchCV(estimator = cbc, param_grid = grid, scoring ='roc_auc', cv = cv, n_jobs=2)

#fit the model
gscv.fit(df_train[using_features], df_train[target])

#returns the estimator with the best performance
print(gscv.best_estimator_)

#returns the best score
print(gscv.best_score_)

#returns the best parameters
print(gscv.best_params_)

0.7059179797275035
{'l2_leaf_reg': 5, 'learning_rate': 0.005, 'max_depth': 6, 'min_child_samples': 4, 'n_estimators': 600}


### Подбор трешхолда

In [340]:
thresholds = np.arange(0.01,0.4,0.05)
score_per_threshold = []

for thold in tqdm_notebook(thresholds):

    scores = []
    for i in range(10):

        features_to_use = features_per_target[target]

        df_train, df_test = train_test_split(train_features, 
                                                 test_size=0.3, 
                                                 shuffle=True, 
                                                 stratify=train_features[target])

        cb = CatBoostClassifier(max_depth=6,
                                n_estimators=600,
                                min_child_samples=4,
                                l2_leaf_reg=5,
                                learning_rate=0.005,
                                verbose=False)
        cb.fit(df_train[features_to_use], df_train[target])

        cb.set_probability_threshold(thold)

        y_pred_cb = cb.predict(df_test[features_to_use])
        score_cb = main_metric(df_test[target], y_pred_cb)

        if np.isnan(score_cb):
            score_cb = 0

        scores.append(score_cb)

    #print(round(np.mean(scores),3))
    
    score_per_threshold.append(round(np.mean(scores),3))
    
pd.DataFrame.from_dict({'threshold' : thresholds,
                        'scores' : score_per_threshold})

  0%|          | 0/8 [00:00<?, ?it/s]

,threshold,scores
0,0.01,0.410
1,0.06,0.472
2,0.11,0.484
3,0.16,0.436
4,0.21,0.219
5,0.26,0.081
6,0.31,0.014
7,0.36,0.011


### Итог

In [227]:
features_target_3 = ['Пол - М',
                     'Семья - в браке в настоящее время',
                     'Семья - в разводе',
                     'Семья - вдовец / вдова',
                     'Семья - гражданский брак / проживание с партнером',
                     'Семья - никогда не был(а) в браке',
                     'Семья - живет один',
                     'Национальность - Русские',
                     'Национальность - Русские / Белорусы / Украинцы',
                     'Религия - Атеист / агностик',
                     'Религия - Христианство',
                     'Религия - Верующий',
                     'Образование - среднее',
                     'Образование - профессиональное училище',
                     'Образование - ВУЗ',
                     'Профессия - ведение домашнего хозяйства',
                     'Профессия - дипломированные специалисты',
                     'Профессия - квалифицированные работники сельского хозяйства и рыболовного',
                     'Профессия - низкоквалифицированные работники',
                     'Профессия - операторы и монтажники установок и машинного оборудования',
                     'Профессия - сфера обслуживания',
                     'Профессия - ремесленники и представители других отраслей промышленности',
                     'Профессия - служащие',
                     'Профессия - техники и младшие специалисты',
                     'Профессия - военные / служащие / управление',
                     'Профессия - промышленность',
                     'Профессия - низкая квалификация',
                     'Вы работаете?',
                     'Выход на пенсию',
                     'Прекращение работы по болезни',
                     'Сахарный диабет',
                     'Гепатит',
                     'Онкология',
                     'Хроническое заболевание легких',
                     'Бронхиальная астма',
                     'Туберкулез легких',
                     'Регулярный прием лекарственных средств',
                     'Травмы за год',
                     'Переломы',
                     'Статус Курения - Бросил(а)',
                     'Статус Курения - Курит',
                     'Статус Курения - Никогда не курил(а)',
                     'Пассивное курение',
                     'Алкоголь - никогда не употреблял',
                     'Алкоголь - ранее употреблял',
                     'Алкоголь - употребляю в настоящее время',
                     'Сон после обеда',
                     'Спорт клубы',
                     'Возраст курения',
                     'Сигарет в день',
                     'Возраст алког',
                     'Образование - Уровень',
                     'Одинокая женщина',
                     'Без работы и не на пенсии',
                     'Сигарет в день (сейчас)',
                     'Частота пасс кур',
                     'Болезнь легких',
                     'Инфекционная болезнь',
                     'Хроническая болезнь',
                     'Регулярный прием лекарственных средств без болезней из опроса',
                     'Больной',
                     'Травмы / переломы',
                     'Идеальное здоровье',
                     'Без вредных привычек',
                     'Бросил вредную привычку',
                     'Макс. возраст вредной привычки',
                     'Активное или пассивное курение',
                     'Продолжительность сна']


target_3_model_params = {'max_depth':6,
                         'n_estimators':600,
                         'min_child_samples':4,
                         'l2_leaf_reg':5,
                         'learning_rate':0.005,
                         'verbose':False}

## Сердечная недостаточность

In [341]:
target = 'Сердечная недостаточность'

In [342]:
using_features = features_per_target[target]

### Поиск по сетке

In [256]:
categorial_cols_using = [x for x in using_features if x in categorial_cols]

#Instantiate CatBoostClassifier
cbc = CatBoostClassifier(verbose=False, 
                         cat_features=categorial_cols_using,
                         )

cv = StratifiedKFold(n_splits=3, shuffle=True)

#create the grid
grid = {'max_depth': [1,2,3],
        'n_estimators':[300,400,500,600],
        'min_child_samples':[4,8,12,16,20],
        'l2_leaf_reg':[0.5,1,5],
        'learning_rate':[0.005,0.01,0.05,0.1]}

#Instantiate GridSearchCV
gscv = GridSearchCV(estimator = cbc, param_grid = grid, scoring ='roc_auc', cv = cv, n_jobs=2)

#fit the model
gscv.fit(df_train[using_features], df_train[target])

#returns the estimator with the best performance
print(gscv.best_estimator_)

#returns the best score
print(gscv.best_score_)

#returns the best parameters
print(gscv.best_params_)

0.554543847241867
{'l2_leaf_reg': 0.5, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_samples': 4, 'n_estimators': 300}


### Подбор трешхолда

In [343]:
thresholds = np.arange(0.01,0.4,0.05)
score_per_threshold = []

for thold in tqdm_notebook(thresholds):

    scores = []
    for i in range(10):

        features_to_use = features_per_target[target]

        df_train, df_test = train_test_split(train_features, 
                                                 test_size=0.3, 
                                                 shuffle=True, 
                                                 stratify=train_features[target])

        cb = CatBoostClassifier(max_depth=3,
                                n_estimators=300,
                                min_child_samples=4,
                                l2_leaf_reg=0.5,
                                learning_rate=0.05,
                                verbose=False)
        cb.fit(df_train[features_to_use], df_train[target])

        cb.set_probability_threshold(thold)

        y_pred_cb = cb.predict(df_test[features_to_use])
        score_cb = main_metric(df_test[target], y_pred_cb)

        if np.isnan(score_cb):
            score_cb = 0

        scores.append(score_cb)

    #print(round(np.mean(scores),3))
    
    score_per_threshold.append(round(np.mean(scores),3))
    
pd.DataFrame.from_dict({'threshold' : thresholds,
                        'scores' : score_per_threshold})

  0%|          | 0/8 [00:00<?, ?it/s]

,threshold,scores
0,0.01,0.357
1,0.06,0.332
2,0.11,0.240
3,0.16,0.127
4,0.21,0.044
5,0.26,0.047
6,0.31,0.049
7,0.36,0.017


### Итог

In [227]:
features_target_4 = ['Возраст алког', 'Макс. возраст вредной привычки', 'Продолжительность сна']


target_4_model_params = {'max_depth':3,
                         'n_estimators':300,
                         'min_child_samples':4,
                         'l2_leaf_reg':0.5,
                         'learning_rate':0.05,
                         'verbose':False}

## Прочие заболевания сердца

In [344]:
target = 'Прочие заболевания сердца'

In [345]:
using_features = features_per_target[target]

### Поиск по сетке

In [259]:
categorial_cols_using = [x for x in using_features if x in categorial_cols]

#Instantiate CatBoostClassifier
cbc = CatBoostClassifier(verbose=False, 
                         cat_features=categorial_cols_using,
                         )

cv = StratifiedKFold(n_splits=3, shuffle=True)

#create the grid
grid = {'max_depth': [5,6,7,8,9,10],
        'n_estimators':[300,400,500,600],
        'min_child_samples':[4,8,12,16,20],
        'l2_leaf_reg':[0.5,1,5],
        'learning_rate':[0.005,0.01,0.05,0.1]}

#Instantiate GridSearchCV
gscv = GridSearchCV(estimator = cbc, param_grid = grid, scoring ='roc_auc', cv = cv, n_jobs=2)

#fit the model
gscv.fit(df_train[using_features], df_train[target])

#returns the estimator with the best performance
print(gscv.best_estimator_)

#returns the best score
print(gscv.best_score_)

#returns the best parameters
print(gscv.best_params_)

0.5871611184292097
{'l2_leaf_reg': 5, 'learning_rate': 0.1, 'max_depth': 10, 'min_child_samples': 4, 'n_estimators': 600}


### Подбор трешхолда

In [346]:
thresholds = np.arange(0.01,0.9,0.05)
score_per_threshold = []

for thold in tqdm_notebook(thresholds):

    scores = []
    for i in range(10):

        features_to_use = features_per_target[target]

        df_train, df_test = train_test_split(train_features, 
                                                 test_size=0.3, 
                                                 shuffle=True, 
                                                 stratify=train_features[target])

        cb = CatBoostClassifier(max_depth=10,
                                n_estimators=600,
                                min_child_samples=4,
                                l2_leaf_reg=5,
                                learning_rate=0.1,
                                verbose=False)
        cb.fit(df_train[features_to_use], df_train[target])

        cb.set_probability_threshold(0.1)

        y_pred_cb = cb.predict(df_test[features_to_use])
        score_cb = main_metric(df_test[target], y_pred_cb)

        if np.isnan(score_cb):
            score_cb = 0

        scores.append(score_cb)

    #print(round(np.mean(scores),3))
    
    score_per_threshold.append(round(np.mean(scores),3))
    
pd.DataFrame.from_dict({'threshold' : thresholds,
                        'scores' : score_per_threshold})

  0%|          | 0/18 [00:00<?, ?it/s]

,threshold,scores
0,0.01,0.204
1,0.06,0.244
2,0.11,0.206
3,0.16,0.215
4,0.21,0.202
5,0.26,0.196
6,0.31,0.174
7,0.36,0.190
8,0.41,0.213
9,0.46,0.181


### Итог

In [281]:
features_target_4 = ['Возраст алког', 'Макс. возраст вредной привычки', 'Продолжительность сна']


target_4_model_params = {'max_depth':10,
                         'n_estimators':600,
                         'min_child_samples':4,
                         'l2_leaf_reg':5,
                         'learning_rate':0.1,
                         'verbose':False}



In [282]:
CatBoostClassifier(target_4_model_params)

# Создадим конфиг для моделей

In [347]:
TARGETS = [
    'Артериальная гипертензия',
    'ОНМК',
    'Стенокардия, ИБС, инфаркт миокарда',
    'Сердечная недостаточность',
    'Прочие заболевания сердца',
]

categorial_cols = ['Пол - М',
 'Семья - в браке в настоящее время',
 'Семья - в разводе',
 'Семья - вдовец / вдова',
 'Семья - гражданский брак / проживание с партнером',
 'Семья - никогда не был(а) в браке',
 'Семья - раздельное проживание (официально не разведены)',
 'Семья - живет один',
 'Этнос - европейская',
 'Этнос - другая азиатская',
 'Этнос - прочее',
 'Национальность - Азербайджанцы',
 'Национальность - Армяне',
 'Национальность - Башкиры',
 'Национальность - Белорусы',
 'Национальность - Буряты',
 'Национальность - Другие национальности',
 'Национальность - Евреи',
 'Национальность - Казахи',
 'Национальность - Киргизы',
 'Национальность - Лезгины',
 'Национальность - Молдаване',
 'Национальность - Мордва',
 'Национальность - Немцы',
 'Национальность - Русские',
 'Национальность - Таджики',
 'Национальность - Татары',
 'Национальность - Удмурты',
 'Национальность - Украинцы',
 'Национальность - Чуваши',
 'Национальность - Эстонцы',
 'Национальность - Азия / Кавказ',
 'Национальность - Русские / Белорусы / Украинцы',
 'Религия - Атеист / агностик',
 'Религия - Другое',
 'Религия - Индуизм',
 'Религия - Ислам',
 'Религия - Нет',
 'Религия - Христианство',
 'Религия - Индуизм / Ислам / Другое',
 'Религия - Верующий',
 'Образование - начальная школа',
 'Образование - среднее',
 'Образование - профессиональное училище',
 'Образование - ВУЗ',
 'Профессия - ведение домашнего хозяйства',
 'Профессия - вооруженные силы',
 'Профессия - дипломированные специалисты',
 'Профессия - квалифицированные работники сельского хозяйства и рыболовного',
 'Профессия - низкоквалифицированные работники',
 'Профессия - операторы и монтажники установок и машинного оборудования',
 'Профессия - законотворцы / менеджеры',
 'Профессия - сфера обслуживания',
 'Профессия - ремесленники и представители других отраслей промышленности',
 'Профессия - служащие',
 'Профессия - техники и младшие специалисты',
 'Профессия - военные / служащие / управление',
 'Профессия - промышленность',
 'Профессия - низкая квалификация',
 'Вы работаете?',
 'Выход на пенсию',
 'Прекращение работы по болезни',
 'Сахарный диабет',
 'Гепатит',
 'Онкология',
 'Хроническое заболевание легких',
 'Бронхиальная астма',
 'Туберкулез легких',
 'ВИЧ/СПИД',
 'Регулярный прием лекарственных средств',
 'Травмы за год',
 'Переломы',
 'Статус Курения - Бросил(а)',
 'Статус Курения - Курит',
 'Статус Курения - Никогда не курил(а)',
 'Пассивное курение',
 'Алкоголь - никогда не употреблял',
 'Алкоголь - ранее употреблял',
 'Алкоголь - употребляю в настоящее время',
 'Сон после обеда',
 'Спорт клубы',
 'Религия клубы',
 'Образование - Уровень',
 'Одинокий мужчина',
 'Одинокая женщина',
 'Без работы и не на пенсии',
 'Болезнь легких',
 'Инфекционная болезнь',
 'Хроническая болезнь',
 'Регулярный прием лекарственных средств без болезней из опроса',
 'Больной',
 'Травмы / переломы',
 'Идеальное здоровье',
 'Без вредных привычек',
 'Бросил вредную привычку',
 'Макс. возраст вредной привычки',
 'Само совершенство',
 'Активное или пассивное курение']


features_per_target = {
 'Артериальная гипертензия': ['Регулярный прием лекарственных средств'],
 'ОНМК': ['Пол - М',
  'Семья - в браке в настоящее время',
  'Семья - вдовец / вдова',
  'Семья - живет один',
  'Национальность - Эстонцы',
  'Образование - среднее',
  'Образование - профессиональное училище',
  'Профессия - дипломированные специалисты',
  'Профессия - низкоквалифицированные работники',
  'Профессия - ремесленники и представители других отраслей промышленности',
  'Профессия - промышленность',
  'Профессия - низкая квалификация',
  'Вы работаете?',
  'Выход на пенсию',
  'Прекращение работы по болезни',
  'Сахарный диабет',
  'Регулярный прием лекарственных средств',
  'Переломы',
  'Статус Курения - Курит',
  'Статус Курения - Никогда не курил(а)',
  'Пассивное курение',
  'Алкоголь - ранее употреблял',
  'Алкоголь - употребляю в настоящее время',
  'Сон после обеда',
  'Спорт клубы',
  'Возраст курения',
  'Сигарет в день',
  'Возраст алког',
  'Образование - Уровень',
  'Одинокая женщина',
  'Без работы и не на пенсии',
  'Сигарет в день (сейчас)',
  'Частота пасс кур',
  'Хроническая болезнь',
  'Регулярный прием лекарственных средств без болезней из опроса',
  'Травмы / переломы',
  'Бросил вредную привычку',
  'Макс. возраст вредной привычки',
  'Активное или пассивное курение',
  'Продолжительность сна'],
 'Стенокардия, ИБС, инфаркт миокарда': ['Пол - М',
  'Семья - в браке в настоящее время',
  'Семья - в разводе',
  'Семья - вдовец / вдова',
  'Семья - гражданский брак / проживание с партнером',
  'Семья - никогда не был(а) в браке',
  'Семья - живет один',
  'Национальность - Русские',
  'Национальность - Русские / Белорусы / Украинцы',
  'Религия - Атеист / агностик',
  'Религия - Христианство',
  'Религия - Верующий',
  'Образование - среднее',
  'Образование - профессиональное училище',
  'Образование - ВУЗ',
  'Профессия - ведение домашнего хозяйства',
  'Профессия - дипломированные специалисты',
  'Профессия - квалифицированные работники сельского хозяйства и рыболовного',
  'Профессия - низкоквалифицированные работники',
  'Профессия - операторы и монтажники установок и машинного оборудования',
  'Профессия - сфера обслуживания',
  'Профессия - ремесленники и представители других отраслей промышленности',
  'Профессия - служащие',
  'Профессия - техники и младшие специалисты',
  'Профессия - военные / служащие / управление',
  'Профессия - промышленность',
  'Профессия - низкая квалификация',
  'Вы работаете?',
  'Выход на пенсию',
  'Прекращение работы по болезни',
  'Сахарный диабет',
  'Гепатит',
  'Онкология',
  'Хроническое заболевание легких',
  'Бронхиальная астма',
  'Туберкулез легких',
  'Регулярный прием лекарственных средств',
  'Травмы за год',
  'Переломы',
  'Статус Курения - Бросил(а)',
  'Статус Курения - Курит',
  'Статус Курения - Никогда не курил(а)',
  'Пассивное курение',
  'Алкоголь - никогда не употреблял',
  'Алкоголь - ранее употреблял',
  'Алкоголь - употребляю в настоящее время',
  'Сон после обеда',
  'Спорт клубы',
  'Возраст курения',
  'Сигарет в день',
  'Возраст алког',
  'Образование - Уровень',
  'Одинокая женщина',
  'Без работы и не на пенсии',
  'Сигарет в день (сейчас)',
  'Частота пасс кур',
  'Болезнь легких',
  'Инфекционная болезнь',
  'Хроническая болезнь',
  'Регулярный прием лекарственных средств без болезней из опроса',
  'Больной',
  'Травмы / переломы',
  'Идеальное здоровье',
  'Без вредных привычек',
  'Бросил вредную привычку',
  'Макс. возраст вредной привычки',
  'Активное или пассивное курение',
  'Продолжительность сна'],
 'Сердечная недостаточность': ['Возраст алког',
  'Макс. возраст вредной привычки',
  'Продолжительность сна'],
 'Прочие заболевания сердца': ['Пол - М',
  'Семья - в браке в настоящее время',
  'Семья - в разводе',
  'Семья - вдовец / вдова',
  'Семья - гражданский брак / проживание с партнером',
  'Семья - никогда не был(а) в браке',
  'Семья - живет один',
  'Этнос - европейская',
  'Этнос - другая азиатская',
  'Этнос - прочее',
  'Национальность - Мордва',
  'Национальность - Немцы',
  'Национальность - Русские',
  'Национальность - Татары',
  'Национальность - Украинцы',
  'Национальность - Чуваши',
  'Национальность - Азия / Кавказ',
  'Национальность - Русские / Белорусы / Украинцы',
  'Религия - Атеист / агностик',
  'Религия - Ислам',
  'Религия - Нет',
  'Религия - Христианство',
  'Религия - Индуизм / Ислам / Другое',
  'Религия - Верующий',
  'Образование - начальная школа',
  'Образование - среднее',
  'Образование - профессиональное училище',
  'Образование - ВУЗ',
  'Профессия - ведение домашнего хозяйства',
  'Профессия - вооруженные силы',
  'Профессия - дипломированные специалисты',
  'Профессия - квалифицированные работники сельского хозяйства и рыболовного',
  'Профессия - низкоквалифицированные работники',
  'Профессия - операторы и монтажники установок и машинного оборудования',
  'Профессия - законотворцы / менеджеры',
  'Профессия - сфера обслуживания',
  'Профессия - ремесленники и представители других отраслей промышленности',
  'Профессия - служащие',
  'Профессия - техники и младшие специалисты',
  'Профессия - военные / служащие / управление',
  'Профессия - промышленность',
  'Профессия - низкая квалификация',
  'Вы работаете?',
  'Выход на пенсию',
  'Прекращение работы по болезни',
  'Сахарный диабет',
  'Гепатит',
  'Онкология',
  'Хроническое заболевание легких',
  'Бронхиальная астма',
  'Туберкулез легких',
  'Регулярный прием лекарственных средств',
  'Травмы за год',
  'Переломы',
  'Статус Курения - Бросил(а)',
  'Статус Курения - Курит',
  'Статус Курения - Никогда не курил(а)',
  'Пассивное курение',
  'Алкоголь - никогда не употреблял',
  'Алкоголь - ранее употреблял',
  'Алкоголь - употребляю в настоящее время',
  'Сон после обеда',
  'Спорт клубы',
  'Религия клубы',
  'Возраст курения',
  'Сигарет в день',
  'Возраст алког',
  'Образование - Уровень',
  'Одинокий мужчина',
  'Одинокая женщина',
  'Без работы и не на пенсии',
  'Сигарет в день (сейчас)',
  'Частота пасс кур',
  'Болезнь легких',
  'Инфекционная болезнь',
  'Хроническая болезнь',
  'Регулярный прием лекарственных средств без болезней из опроса',
  'Больной',
  'Травмы / переломы',
  'Идеальное здоровье',
  'Без вредных привычек',
  'Бросил вредную привычку',
  'Макс. возраст вредной привычки',
  'Само совершенство',
  'Активное или пассивное курение',
  'Продолжительность сна']}


model_params_per_target = {
    'Артериальная гипертензия' : {'max_depth':1,
                         'n_estimators':300,
                         'min_child_samples':4,
                         'l2_leaf_reg':0.5,
                         'learning_rate':0.005,
                         'verbose':False},
    'ОНМК' : {'max_depth':5,
                         'n_estimators':500,
                         'min_child_samples':4,
                         'l2_leaf_reg':5,
                         'learning_rate':0.1,
                         'verbose':False},
    'Стенокардия, ИБС, инфаркт миокарда' : {'max_depth':6,
                         'n_estimators':600,
                         'min_child_samples':4,
                         'l2_leaf_reg':5,
                         'learning_rate':0.005,
                         'verbose':False},
    'Сердечная недостаточность' : {'max_depth':3,
                         'n_estimators':300,
                         'min_child_samples':4,
                         'l2_leaf_reg':0.5,
                         'learning_rate':0.05,
                         'verbose':False},
    'Прочие заболевания сердца' : {'max_depth':10,
                         'n_estimators':600,
                         'min_child_samples':4,
                         'l2_leaf_reg':5,
                         'learning_rate':0.1,
                         'verbose':False}
}

model_threshold_per_target = {
    'Артериальная гипертензия' : 0.2,
    'ОНМК' : 0.01,
    'Стенокардия, ИБС, инфаркт миокарда' : 0.11,
    'Сердечная недостаточность' : 0.01,
    'Прочие заболевания сердца' : 0.61
}

In [284]:
cb.save_model('model_test',
           format="cbm",
           export_parameters=None,
           pool=None)

In [354]:
model_params_per_target[target]

{'max_depth': 1,
 'n_estimators': 300,
 'min_child_samples': 4,
 'l2_leaf_reg': 0.5,
 'learning_rate': 0.005,
 'verbose': False}

In [361]:
scores_cb3 = []

for target in tqdm_notebook(TARGETS):
    
    scores = []
    for i in range(20):
        
        features_to_use = features_per_target[target]
        
        df_train, df_test = train_test_split(train_features, 
                                                 test_size=0.3, 
                                                 shuffle=True, 
                                                 stratify=train_features[target])

        cb = CatBoostClassifier(**model_params_per_target[target])
        cb.fit(df_train[features_to_use], df_train[target])
        
        cb.set_probability_threshold(model_threshold_per_target[target])
        
        y_pred_cb = cb.predict(df_test[features_to_use])
        score_cb = main_metric(df_test[target], y_pred_cb)

        if np.isnan(score_cb):
            score_cb = 0

        scores.append(score_cb)
        
    scores_cb3.append(round(np.mean(scores), 3))

  0%|          | 0/5 [00:00<?, ?it/s]

In [362]:
scores_cb3.append(np.mean(scores_cb3))

In [363]:
indexes = []
for target in TARGETS:
    indexes.append(f'Score - {target}')
    
indexes.append('Average Score ALL')


df_res = pd.DataFrame.from_dict({'LogReg' : lr_data,
                                 'RandFor' : rf_data,
                                 'CatBoost' : cb_data,
                                 'CatBoost + features' : scores_cb2,
                                 'CatBoost final' : scores_cb3})

df_res.index = indexes

df_res

,LogReg,RandFor,CatBoost,CatBoost + features,CatBoost final
Score - Артериальная гипертензия,0.702,0.708,0.718,0.809,0.8140
Score - ОНМК,0.016,0.000,0.004,0.005,0.2200
"Score - Стенокардия, ИБС, инфаркт миокарда",0.052,0.007,0.004,0.019,0.5060
Score - Сердечная недостаточность,0.034,0.007,0.000,0.010,0.3550
Score - Прочие заболевания сердца,0.000,0.000,0.000,0.007,0.0020
Average Score ALL,0.161,0.144,0.145,0.170,0.3794


Отлично. Финальная модель оказалась больше чем в два раза лучше.